In [1]:
# Load environment variables
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

import yaml
from crewai import Agent, Task, Crew, LLM

In [2]:
# llm = LLM(
#     model="groq/deepseek-r1-distill-llama-70b",
#     temperature=0.7,  # Adjust as needed
# )

llm = LLM(
    model="gemini/gemini-2.0-flash-thinking-exp-01-21",
    temperature=0.7
)


In [3]:
# Define file paths for YAML configurations relative to the script's directory
files = {
    'agents': r'C:\Users\Admin\Pictures\c\AGENT\Test_9\config\agents.yaml',
    'tasks': r'C:\Users\Admin\Pictures\c\AGENT\Test_9\config\tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [4]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

In [5]:
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent'],
  llm=llm
)

estimation_agent = Agent(
  config=agents_config['estimation_agent'],
  llm=llm
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent'],
  llm=llm
)

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

LLM value is already an LLM object
LLM value is already an LLM object
LLM value is already an LLM object


In [6]:
from IPython.display import display, Markdown

project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



In [7]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

In [8]:
# Run the crew
result = crew.kickoff(
  inputs=inputs
)

# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build tru

In [9]:

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

NameError: name 'df_tasks' is not defined

In [12]:
import pandas as pd
tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)
df_tasks


,task_name,estimated_time_hours,required_resources
0,Project Kick-off Meeting,8.0,[John Doe (Project Manager)]
1,Requirements Gathering & Analysis,16.0,[John Doe (Project Manager)]
2,Sitemap & Content Outline Creation,24.0,"[John Doe (Project Manager), Bob Smith (Design..."
3,UI/UX Design - Wireframing,24.0,[Bob Smith (Designer)]
4,UI/UX Design - Mockups & Style Guide,40.0,[Bob Smith (Designer)]
5,Design Review & Approval,8.0,"[John Doe (Project Manager), Bob Smith (Design..."
6,Set up Development Environment,8.0,[Jane Doe (Software Engineer)]
7,Front-end - Base & Responsive,24.0,[Jane Doe (Software Engineer)]
8,Front-end - Page Implementation (All Pages),56.0,[Jane Doe (Software Engineer)]
9,Back-end - Blog Functionality (if dynamic),40.0,[Jane Doe (Software Engineer)]


In [2]:
# experiment.ipynb
# Import necessary modules
from config_loader import ConfigLoader
from llm_setup import LLMSetup
from crew_setup import CrewSetup
from models import ProjectPlan
from utils import DisplayUtils

# Initialize components
config_loader = ConfigLoader()
configs = config_loader.load_configs()
llm = LLMSetup.get_llm()

# Setup crew
crew_setup = CrewSetup(configs['agents'], configs['tasks'], llm)
agents = crew_setup.create_agents()
tasks = crew_setup.create_tasks(agents)
crew = crew_setup.assemble_crew(agents, tasks)

# Prepare and display project details
project_data = {
    'project': 'Website',
    'industry': 'Technology',
    'project_objectives': 'Create a website for a small business',
    'team_members': """
    - John Doe (Project Manager)
    - Jane Doe (Software Engineer)
    - Bob Smith (Designer)
    - Alice Johnson (QA Engineer)
    - Tom Brown (QA Engineer)
    """,
    'project_requirements': """
    - Create a responsive design that works well on desktop and mobile devices
    - Implement a modern, visually appealing user interface with a clean look
    - Develop a user-friendly navigation system with intuitive menu structure
    - Include an "About Us" page highlighting the company's history and values
    - Design a "Services" page showcasing the business's offerings with descriptions
    - Create a "Contact Us" page with a form and integrated map for communication
    - Implement a blog section for sharing industry news and company updates
    - Ensure fast loading times and optimize for search engines (SEO)
    - Integrate social media links and sharing capabilities
    - Include a testimonials section to showcase customer feedback and build trust
    """
}

DisplayUtils.show_markdown(DisplayUtils.format_project_details(project_data))

# Run the crew
inputs = {
    'project_type': project_data['project'],
    'project_objectives': project_data['project_objectives'],
    'industry': project_data['industry'],
    'team_members': project_data['team_members'],
    'project_requirements': project_data['project_requirements']
}

result = crew.kickoff(inputs=inputs)

ModuleNotFoundError: No module named 'config_loader'